# SCREENER

Importing packages

In [96]:


# ========================
# Data Import and Handling
# ========================
import pandas as pd  # For data manipulation and analysis
import requests  # For making HTTP requests
import os

# =======================
# Date and Time Handling
# =======================
from datetime import datetime, timedelta  # For working with dates and times

# ==============================
# Technical Analysis Indicators
# ==============================
import talib  # For technical analysis indicators

# ===================
# Plotting with Matplotlib
# ===================
import matplotlib.pyplot as plt  # For plotting using Matplotlib

# ======================
# Interactive Plotting
# ======================
import plotly.graph_objects as go  # For interactive plotting with Plotly
import plotly.io as pio

# ============================
# Statistical Data Visualization
# ============================
import seaborn as sns  # For statistical data visualization

# ==============================
# Displaying HTML in IPython
# ==============================
from IPython.display import HTML  

# ===============================
# Financial Charting with mplfinance
# ===============================
import mplfinance as mpf  # For financial charting with Matplotlib

# ===============================
# Email
# ===============================
import smtplib, ssl


In [97]:
API_URL = 'https://api.binance.com/api/v3/klines' # Binance API endpoint for candlestick data

## Functions

In [120]:
# ==============================
# RETRIEVING DATA FROM API
# ==============================

#The variable days_back represents the number of days in the past from which I intend to retrieve the data.
#The variable symbol is a coin pair. For example if you want bitcoin historical data, the symbol to use is BTCUSDT. 


def get_historical_data(symbol, days_back, interval):
    # Calculate the start time in milliseconds
    start_time = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)

    # Calculate the current time in milliseconds
    end_time = int(datetime.now().timestamp() * 1000)

    # Make an API request to retrieve historical data
    response = requests.get(API_URL, params={
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Maximum number of data points per request
    })

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # API request was successful
        hist_json = response.json()

        # Making a df from response
        # Extract only the relevant elements for each row
        df = pd.DataFrame(hist_json, columns=[
            'Time', 'Open Price', 'High Price', 'Low Price',
            'Close Price', 'Volume', 'Kline Close Time', 'Quote Asset Volume',
            'Number of Trades', 'Taker Buy Base Asset Volume',
            'Taker Buy Quote Asset Volume', 'Unused Field'
        ])

        # Select only the relevant columns (Open Time, Open Price, High Price, Low Price, Close Price, Volume)
        df = df[['Time', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']]
        df['dateTime'] = pd.to_datetime(df['Time'], unit='ms')
        print("Retrived data for: " + symbol)

        return df
    else:
        # API request was not successful, print an error message
        print(f"Error: API request failed with status code {response.status_code}")
        return None

# ==============================
# TRANSFORM DATA
# ==============================

def transform_hist_data(df_name):
    df_name['Open Price'] = pd.to_numeric(df_name['Open Price'], errors='coerce').astype(float)
    df_name['High Price'] = pd.to_numeric(df_name['High Price'], errors='coerce').astype(float)
    df_name['Low Price'] = pd.to_numeric(df_name['Low Price'], errors='coerce').astype(float)
    df_name['Close Price'] = pd.to_numeric(df_name['Close Price'], errors='coerce').astype(float)
    df_name['Volume'] = pd.to_numeric(df_name['Volume'], errors='coerce').astype(float)
    return df_name

# ==============================
# CALCULATE EMA 12, 13 and 26
# ==============================
def calculate_emas(df_name):
    df_name['13EMA'] = df_name['Close Price'].ewm(span=13, adjust=False).mean()
    df_name['12EMA'] = df_name['Close Price'].ewm(span=12, adjust=False).mean()
    df_name['26EMA'] = df_name['Close Price'].ewm(span=26, adjust=False).mean()
    return df_name
    
# ==============================
# CALCULATE MACD 12, 26, 9
# ==============================

def calculate_macd(df_name):
    # Calculate MACD Line (12EMA - 26EMA)
    df_name['MACD'] = df_name['12EMA'] - df_name['26EMA']

    # Calculate Signal Line (9-period EMA of MACD)
    df_name['Signal'] = df_name['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate MACD Histogram
    df_name['Histogram'] = df_name['MACD'] - df_name['Signal']
    return df_name
    
# ==============================
# CALCULATE ELDER IMPULSE
# ==============================

def calculate_elder(df_name):

    for i in range(1, len(df_name)):
        ema_comparison = 0
        macd_comparison = 0
        first_row_ema = df_name.iloc[i-1]['13EMA']
        second_row_ema = df_name.iloc[i]['13EMA']
        if second_row_ema > first_row_ema:
            ema_comparison = 1
        elif second_row_ema < first_row_ema:
            ema_comparison = -1
        first_row_macd = df_name.iloc[i-1]['Histogram']
        second_row_macd = df_name.iloc[i]['Histogram']
        if second_row_macd > first_row_macd:
            macd_comparison = 1
        elif second_row_macd < first_row_macd:
            macd_comparison = -1
        if ema_comparison == 1 and macd_comparison == 1:
              df_name.at[i, 'elder_impulse'] = 1
        elif ema_comparison == -1 and macd_comparison == -1:
            df_name.at[i, 'elder_impulse'] = -1
        else:
            df_name.at[i, 'elder_impulse'] = 0
    return df_name
# ==============================
# CALCULATE ATR CHANNELS
# ==============================
def calculate_ATR(df_name):
    df_name['ATR21'] = talib.ATR(df_name['High Price'], df_name['Low Price'], df_name['Close Price'], timeperiod=21)
    df_name['21EMA'] = df_name['Close Price'].ewm(span=21, adjust=False).mean()
    df_name['ATR+1'] = df_name['21EMA'] + df_name['ATR21'] 
    df_name['ATR+2'] = df_name['21EMA'] + 2 * df_name['ATR21'] 
    df_name['ATR+3'] = df_name['21EMA'] + 3 * df_name['ATR21'] 
    df_name['ATR-1'] = df_name['21EMA'] - df_name['ATR21'] 
    df_name['ATR-2'] = df_name['21EMA'] - 2 * df_name['ATR21'] 
    df_name['ATR-3'] = df_name['21EMA'] - 3 * df_name['ATR21'] 
    return df_name
# ==============================
# TOP CRYPTO TO USDT LIST
# ==============================

def get_top_cryptos_with_usdt(number = 30):
    url = "https://api.binance.com/api/v3/ticker/24hr"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        symbol_count_dict = {entry.get('symbol', None): entry.get('count', None) for entry in data if 'USDT' in entry.get('symbol', '')}
        sorted_dict_values = dict(sorted(symbol_count_dict.items(), key=lambda item: item[1], reverse = True))
        top_crypto_usdt = list(sorted_dict_values.keys())[:number]
        return top_crypto_usdt
    else:
        print(f"Error: {response.status_code}")
        return None


# ==============================
# DRAW ATR GRAPH
# ==============================

def draw_ATR_GRAPH(df_name):


    # Create a candlestick trace
    df_name['formatted_date'] = df_name['dateTime'].dt.strftime('%m-%d')
    candlestick = go.Candlestick(x=df_name['formatted_date'],
                                open=df_name['Open Price'],
                                high=df_name['High Price'],
                                low=df_name['Low Price'],
                                close=df_name['Close Price'],
                                name='Candlestick')

    # 21 EMA trace
    line_trace_21_EMA = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['21EMA'],
                        mode='lines',
                        name='21EMA',
                        line=dict(color= 'black'))
    # ATR +1 trace
    line_trace_ATR_1 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+1'],
                        mode='lines',
                        name='ATR+1',
                        line=dict(color= 'green'))

    # ATR +2 trace
    line_trace_ATR_2 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+2'],
                        mode='lines',
                        name='ATR+2',
                        line=dict(color= 'orange'))


    # ATR +3 trace
    line_trace_ATR_3 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+3'],
                        mode='lines',
                        name='ATR+3',
                        line=dict(color= 'red'))

    # ATR -1 trace
    line_trace_ATR_neg1 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-1'],
                        mode='lines',
                        name='ATR-1',
                        line=dict(color= 'green'))
    # ATR -2 trace
    line_trace_ATR_neg2 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-2'],
                        mode='lines',
                        name='ATR-2',
                        line=dict(color= 'orange'))
    # ATR -3 trace
    line_trace_ATR_neg3 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-3'],
                        mode='lines',
                        name='ATR-3',
                        line=dict(color= 'red'))

    # Create layout
    layout = go.Layout(title='Candlestick Chart with Extra Line',
                    xaxis=dict(type='category', categoryorder='category ascending'),
                    yaxis=dict(title='Price'))

    # Create figure

    fig = go.Figure(data=[candlestick, line_trace_21_EMA, line_trace_ATR_1, line_trace_ATR_2, line_trace_ATR_3, line_trace_ATR_neg1, line_trace_ATR_neg2, line_trace_ATR_neg3], layout=layout)
    return fig

# ==============================
# CREATE BEAR CHARTS and put them into a figures folder
# ==============================

def create_bear_charts():

    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')

    # Specify the folder path
    folder_path = f'/Users/ahtojarve/Trading_bot/reports/figures/{today_date}/bear'

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'Folder "{folder_path}" created successfully.')
    else:
        print(f'Folder "{folder_path}" already exists.')

   # Create an empty DataFrame

    for crypto in bear_set:
        df_medium = get_historical_data(crypto, 60, '1d')
        if len(df_medium) < 60:
            print(f"Skipping {crypto} - Insufficient historical data (less than 60 rows).")
            continue
        df_medium = calculate_ATR(df_medium)
        df_medium = transform_hist_data(df_medium)
        fig = draw_ATR_GRAPH(df_medium)
        chart_filename = f"{folder_path}/{crypto}.html"
        pio.write_html(fig, file = chart_filename)

# ==============================
# CREATE BULL CHARTS and put them into a figures folder
# ==============================

def create_bull_charts():

    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')

    # Specify the folder path
    folder_path = f'/Users/ahtojarve/Trading_bot/reports/figures/{today_date}/bull'

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'Folder "{folder_path}" created successfully.')
    else:
        print(f'Folder "{folder_path}" already exists.')

    for crypto in bull_set:
        df_medium = get_historical_data(crypto, 60, '1d')
        if len(df_medium) < 60:
            print(f"Skipping {crypto} - Insufficient historical data (less than 60 rows).")
            continue
        df_medium = calculate_ATR(df_medium)
        df_medium = transform_hist_data(df_medium)
        fig = draw_ATR_GRAPH(df_medium)
        chart_filename = f"{folder_path}/{crypto}.html"
        pio.write_html(fig, file = chart_filename)


## Creating a table with elder impulse

Elder Impulse
https://school.stockcharts.com/doku.php?id=chart_analysis:elder_impulse_system
MACD-histogram
https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp

In [13]:
df_long = get_historical_data('BTCUSDT', 360, '1w')
df_long = transform_hist_data(df_long)
df_long = calculate_emas(df_long)
df_long = calculate_macd(df_long)
df_long = calculate_elder(df_long)

Retrived data for: BTCUSDT


In [14]:
df_long.tail(10)

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA,12EMA,26EMA,MACD,Signal,Histogram,elder_impulse
42,1704067200000,42283.58,45879.63,40750.00,43929.02,310487.48622,2024-01-01,38590.195710,38995.953657,35008.801205,3987.152452,2903.195983,1083.956469,0.0
43,1704672000000,43929.01,48969.48,41500.00,41732.35,470798.80434,2024-01-08,39039.074894,39416.937710,35506.841857,3910.095853,3104.575957,805.519896,0.0
44,1705276800000,41732.35,43578.01,40280.00,41580.33,238486.27274,2024-01-15,39402.111338,39749.767293,35956.729867,3793.037426,3242.268250,550.769175,0.0
45,1705881600000,41580.32,42842.68,38555.00,42031.06,274603.16207,2024-01-22,39777.675432,40100.735402,36406.680247,3694.055154,3332.625631,361.429523,0.0
46,1706486400000,42031.05,43882.36,41804.88,42582.88,203036.61925,2024-01-29,40178.418942,40482.603801,36864.176525,3618.427276,3389.785960,228.641316,0.0
47,1707091200000,42582.88,48592.66,42258.10,48299.99,262240.48357,2024-02-05,41338.643379,41685.278601,37711.273820,3974.004781,3506.629724,467.375057,1.0
48,1707696000000,48300.00,52816.62,47710.01,52137.67,310862.30170,2024-02-12,42881.361468,43293.338816,38779.895759,4513.443057,3707.992391,805.450666,1.0
49,1708300800000,52137.68,52985.00,50521.00,51728.85,223366.16186,2024-02-19,44145.288401,44591.109768,39739.077555,4852.032213,3936.800355,915.231858,1.0
50,1708905600000,51728.85,64000.00,50901.44,63113.97,417907.83383,2024-02-26,46855.100058,47440.780573,41470.551069,5970.229504,4343.486185,1626.743319,1.0
51,1709510400000,63113.97,67524.99,62300.00,67335.70,69899.23920,2024-03-04,49780.900050,50501.537408,43386.488027,7115.049381,4897.798824,2217.250557,1.0


if elder_impulse is 1, it means you can go long; 
if it is -1 , you can go short;
if it is 0 you can do both. 
It is best to go long if impulse is 1 or 0 after -1. (This should be considered good metricks for stock screening)

# SCREENER

- Leian top krüptod
- Laen iga krüpto kohta alla pika vaate
- Teen EMA ja MACD arvutused
- Teen Elder Impulse arvutused
- Värviliselt näitan ainult neid, kus peale punast tuleb roheline või sinine või siis peale rohelist tuleb sinine või punane


## Leian top krüptod

In [99]:


# Define colors
green_color = "#00cc00"
red_color = "#ff6666"

bull_set = set() #Creating a set for all bulls
bear_set = set() #Creating a set for all bears

top_cryptos_usdt = get_top_cryptos_with_usdt(100)

for crypto in top_cryptos_usdt:
    #Teen arvutused ja panen tabelisse
    df = get_historical_data(crypto, 360, '1w')
    df = transform_hist_data(df)
    df = calculate_emas(df)
    df = calculate_macd(df)
    df = calculate_elder(df)
    #Kui elder on roheline või sinine peale punast, siis on pull
    if len(df) >=2:

        if df.iloc[-2]['elder_impulse'] == -1 and (df.iloc[-1]['elder_impulse'] == 0 or df.iloc[-1]['elder_impulse'] == 1):
            bull_set.add(crypto)
            display(HTML(f"<span style='color:{green_color}'>Added {crypto} to bull_set</span>"))
        #Kui elder on punane või sinine peale rohelist, siis on karu
        elif df.iloc[-2]['elder_impulse'] == 1 and (df.iloc[-1]['elder_impulse'] == 0 or df.iloc[-1]['elder_impulse'] == -1):
            bear_set.add(crypto)
            display(HTML(f"<span style='color:{red_color}'>Added {crypto} to bear_set</span>"))
        else:
            print(crypto + " is not acceptable")
    else:
        print(crypto + " is not acceptable")
    time.sleep(0.1)
    
    
    

Retrived data for: BTCUSDT
BTCUSDT is not acceptable
Retrived data for: FDUSDUSDT


Retrived data for: SHIBUSDT
SHIBUSDT is not acceptable
Retrived data for: SOLUSDT
SOLUSDT is not acceptable
Retrived data for: FETUSDT
FETUSDT is not acceptable
Retrived data for: ETHUSDT
ETHUSDT is not acceptable
Retrived data for: PEPEUSDT
PEPEUSDT is not acceptable
Retrived data for: AGIXUSDT
AGIXUSDT is not acceptable
Retrived data for: RNDRUSDT
RNDRUSDT is not acceptable
Retrived data for: DOGEUSDT
DOGEUSDT is not acceptable
Retrived data for: FLOKIUSDT
FLOKIUSDT is not acceptable
Retrived data for: NEARUSDT
NEARUSDT is not acceptable
Retrived data for: NFPUSDT
NFPUSDT is not acceptable
Retrived data for: ANKRUSDT
ANKRUSDT is not acceptable
Retrived data for: PHBUSDT
PHBUSDT is not acceptable
Retrived data for: FTMUSDT
FTMUSDT is not acceptable
Retrived data for: BONKUSDT
BONKUSDT is not acceptable
Retrived data for: AIUSDT
AIUSDT is not acceptable
Retrived data for: WLDUSDT


Retrived data for: RSRUSDT
RSRUSDT is not acceptable
Retrived data for: MDTUSDT
MDTUSDT is not acceptable
Retrived data for: JUPUSDT
JUPUSDT is not acceptable
Retrived data for: LUNAUSDT
LUNAUSDT is not acceptable
Retrived data for: LUNCUSDT
LUNCUSDT is not acceptable
Retrived data for: BNBUSDT
BNBUSDT is not acceptable
Retrived data for: XAIUSDT
XAIUSDT is not acceptable
Retrived data for: MANTAUSDT
MANTAUSDT is not acceptable
Retrived data for: SEIUSDT
SEIUSDT is not acceptable
Retrived data for: ARUSDT
ARUSDT is not acceptable
Retrived data for: ORDIUSDT


Retrived data for: XRPUSDT
XRPUSDT is not acceptable
Retrived data for: OCEANUSDT
OCEANUSDT is not acceptable
Retrived data for: WIFUSDT
WIFUSDT is not acceptable
Retrived data for: PIXELUSDT


Retrived data for: ALTUSDT
ALTUSDT is not acceptable
Retrived data for: MATICUSDT
MATICUSDT is not acceptable
Retrived data for: FILUSDT
FILUSDT is not acceptable
Retrived data for: APTUSDT
APTUSDT is not acceptable
Retrived data for: ARBUSDT
ARBUSDT is not acceptable
Retrived data for: ARKMUSDT
ARKMUSDT is not acceptable
Retrived data for: MEMEUSDT
MEMEUSDT is not acceptable
Retrived data for: JASMYUSDT
JASMYUSDT is not acceptable
Retrived data for: GRTUSDT
GRTUSDT is not acceptable
Retrived data for: SUIUSDT
SUIUSDT is not acceptable
Retrived data for: 1000SATSUSDT
1000SATSUSDT is not acceptable
Retrived data for: GTCUSDT
GTCUSDT is not acceptable
Retrived data for: INJUSDT
INJUSDT is not acceptable
Retrived data for: RUNEUSDT
RUNEUSDT is not acceptable
Retrived data for: FIDAUSDT
FIDAUSDT is not acceptable
Retrived data for: JTOUSDT
JTOUSDT is not acceptable
Retrived data for: ATOMUSDT
ATOMUSDT is not acceptable
Retrived data for: DOTUSDT
DOTUSDT is not acceptable
Retrived data for:

Retrived data for: USDTTRY
USDTTRY is not acceptable
Retrived data for: RAYUSDT
RAYUSDT is not acceptable
Retrived data for: IDUSDT
IDUSDT is not acceptable
Retrived data for: AVAXUSDT
AVAXUSDT is not acceptable
Retrived data for: LPTUSDT
LPTUSDT is not acceptable
Retrived data for: FISUSDT
FISUSDT is not acceptable
Retrived data for: LINKUSDT


Retrived data for: DEXEUSDT
DEXEUSDT is not acceptable
Retrived data for: ICPUSDT
ICPUSDT is not acceptable
Retrived data for: ZRXUSDT
ZRXUSDT is not acceptable
Retrived data for: OPUSDT
OPUSDT is not acceptable
Retrived data for: MTLUSDT
MTLUSDT is not acceptable
Retrived data for: THETAUSDT
THETAUSDT is not acceptable
Retrived data for: MOVRUSDT
MOVRUSDT is not acceptable
Retrived data for: UNIUSDT
UNIUSDT is not acceptable
Retrived data for: FTTUSDT
FTTUSDT is not acceptable
Retrived data for: ALGOUSDT
ALGOUSDT is not acceptable
Retrived data for: GALAUSDT
GALAUSDT is not acceptable
Retrived data for: CFXUSDT
CFXUSDT is not acceptable
Retrived data for: ROSEUSDT
ROSEUSDT is not acceptable
Retrived data for: BEAMXUSDT
BEAMXUSDT is not acceptable
Retrived data for: SUPERUSDT
SUPERUSDT is not acceptable
Retrived data for: COTIUSDT
COTIUSDT is not acceptable
Retrived data for: CKBUSDT
CKBUSDT is not acceptable
Retrived data for: ONEUSDT
ONEUSDT is not acceptable
Retrived data for: AAVEU

Retrived data for: DYDXUSDT
DYDXUSDT is not acceptable
Retrived data for: PROMUSDT
PROMUSDT is not acceptable
Retrived data for: KEYUSDT
KEYUSDT is not acceptable
Retrived data for: SANDUSDT
SANDUSDT is not acceptable
Retrived data for: SPELLUSDT
SPELLUSDT is not acceptable
Retrived data for: ACHUSDT
ACHUSDT is not acceptable
Retrived data for: CRVUSDT
CRVUSDT is not acceptable
Retrived data for: FLOWUSDT
FLOWUSDT is not acceptable
Retrived data for: CYBERUSDT
CYBERUSDT is not acceptable
Retrived data for: BCHUSDT
BCHUSDT is not acceptable
Retrived data for: ORNUSDT
ORNUSDT is not acceptable
Retrived data for: HOTUSDT
HOTUSDT is not acceptable
Retrived data for: DYMUSDT


Retrived data for: TIAUSDT
TIAUSDT is not acceptable
Retrived data for: HBARUSDT
HBARUSDT is not acceptable
Retrived data for: CHESSUSDT
CHESSUSDT is not acceptable
Retrived data for: CELOUSDT
CELOUSDT is not acceptable


## Creating Medium view and ATR channels

In [113]:
df_medium = get_historical_data('BTCUSDT', 60, '1d')
df_medium = calculate_ATR(df_medium)
df_medium = transform_hist_data(df_medium)

Retrived data for: BTCUSDT


In [52]:
draw_ATR_GRAPH(df_medium)

Siit lihtsalt testimine. Salvestab ilusti htmlina ära. 
Siit edasi:
1. tee tsükkel, kus teeb html-id nendest, mis on pullid ja bearid
2. saada need meilile

In [54]:

# Output directory where HTML files will be saved
output_directory = '/Users/ahtojarve/Trading_bot/reports/figures/'
os.makedirs(output_directory, exist_ok=True)
chart_filename = f"{output_directory}testing.html"
pio.write_html(fig, file = chart_filename)

## EMAILi saatmine

Praegu ei tööta see hästi, peab edasi vaatama, mis teha ja kas see emaili saatmine on üldse vajalik. 
Kui teha dashboard ei ole kirjal vist pointi. 

In [63]:
'''file_path = '/Users/ahtojarve/Trading_bot/reports/figures/testing.html'
html_string = ''

with open(file_path, 'r', encoding='utf-8') as file:
    html_string = file.read()

print(f'HTML content loaded from "{file_path}" into a string variable.')
'''

HTML content loaded from "/Users/ahtojarve/Trading_bot/reports/figures/testing.html" into a string variable.


In [64]:
'''

import yagmail

receiver = "ahtotrade@gmail.com"
body = html_string

yag = yagmail.SMTP(sender_email, "uftcphhmfxeuggyo")
yag.send(
    to=receiver,
    subject="Yagmail test with attachment",
    contents=body, 
    
)

'''

{}

## Loop graafikute joonistamiseks

Ma teen praegu kõik kohaliku arvutiga. Hiljem on vaja teha ümber, panna kuhugi dashboardile või näiteks google drivei

In [121]:
create_bear_charts()

Folder "/Users/ahtojarve/Trading_bot/reports/figures/2024-03-07/bear" created successfully.
Retrived data for: LINKUSDT
Retrived data for: IMXUSDT
Retrived data for: ORDIUSDT
Retrived data for: WLDUSDT


In [122]:
create_bull_charts()

Folder "/Users/ahtojarve/Trading_bot/reports/figures/2024-03-07/bull" created successfully.
Retrived data for: DYMUSDT
Skipping DYMUSDT - Insufficient historical data (less than 60 rows).
Retrived data for: STRKUSDT
Skipping STRKUSDT - Insufficient historical data (less than 60 rows).
Retrived data for: FDUSDUSDT
Retrived data for: PIXELUSDT
Skipping PIXELUSDT - Insufficient historical data (less than 60 rows).


In [107]:
df_medium.tail(10)

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,21EMA,ATR+1,ATR+2,ATR+3,ATR-1,ATR-2,ATR-3,formatted_date
50,1708819200000,0.03627,0.03782,0.03552,0.03759,32987522.0,2024-02-25,0.003120,0.034646,0.037767,0.040887,0.044007,0.031526,0.028406,0.025285,02-25
51,1708905600000,0.03763,0.03778,0.03475,0.03550,84762075.0,2024-02-26,0.003116,0.034724,0.037840,0.040956,0.044072,0.031608,0.028492,0.025376,02-26
52,1708992000000,0.03550,0.03613,0.03479,0.03504,71040577.0,2024-02-27,0.003031,0.034753,0.037784,0.040815,0.043847,0.031721,0.028690,0.025658,02-27
53,1709078400000,0.03508,0.03609,0.03322,0.03424,72769617.0,2024-02-28,0.003024,0.034706,0.037730,0.040754,0.043777,0.031682,0.028658,0.025635,02-28
54,1709164800000,0.03424,0.03980,0.03390,0.03810,164931172.0,2024-02-29,0.003161,0.035015,0.038175,0.041336,0.044497,0.031854,0.028693,0.025532,02-29
55,1709251200000,0.03811,0.03926,0.03660,0.03768,117816371.0,2024-03-01,0.003137,0.035257,0.038394,0.041531,0.044667,0.032120,0.028983,0.025846,03-01
56,1709337600000,0.03768,0.03889,0.03700,0.03870,68561689.0,2024-03-02,0.003078,0.035570,0.038647,0.041725,0.044802,0.032492,0.029415,0.026337,03-02
57,1709424000000,0.03869,0.05199,0.03645,0.04734,521781107.0,2024-03-03,0.003671,0.036640,0.040311,0.043982,0.047653,0.032969,0.029298,0.025627,03-03
58,1709510400000,0.04734,0.04757,0.04030,0.04265,271056447.0,2024-03-04,0.003842,0.037186,0.041029,0.044871,0.048713,0.033344,0.029502,0.025659,03-04
59,1709596800000,0.04262,0.04346,0.03650,0.03861,147350018.0,2024-03-05,0.003991,0.037316,0.041306,0.045297,0.049288,0.033325,0.029334,0.025343,03-05


## TROUBLESHOOTING WEIRD CHARTS

In [114]:
df_medium = get_historical_data('PIXELUSDT', 60, '1d')

Retrived data for: PIXELUSDT


In [115]:
df_medium.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Time         18 non-null     int64         
 1   Open Price   18 non-null     object        
 2   High Price   18 non-null     object        
 3   Low Price    18 non-null     object        
 4   Close Price  18 non-null     object        
 5   Volume       18 non-null     object        
 6   dateTime     18 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.1+ KB


In [116]:
df_medium

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime
0,1708300800000,0.04000000,0.69470000,0.04000000,0.54750000,1427441771.50000000,2024-02-19
1,1708387200000,0.54750000,0.61780000,0.48160000,0.51600000,630112680.10000000,2024-02-20
2,1708473600000,0.51600000,0.54790000,0.48120000,0.51890000,251530207.30000000,2024-02-21
3,1708560000000,0.51890000,0.56760000,0.49010000,0.52320000,221143329.80000000,2024-02-22
4,1708646400000,0.52330000,0.54970000,0.49030000,0.50100000,137532554.30000000,2024-02-23
5,1708732800000,0.50090000,0.52960000,0.48100000,0.52000000,81988227.10000000,2024-02-24
6,1708819200000,0.52020000,0.58800000,0.50270000,0.56460000,152640789.20000000,2024-02-25
7,1708905600000,0.56490000,0.64000000,0.55660000,0.56940000,242845486.70000000,2024-02-26
8,1708992000000,0.56930000,0.60480000,0.54100000,0.56070000,151172669.70000000,2024-02-27
9,1709078400000,0.56080000,0.56800000,0.45000000,0.50960000,202701076.90000000,2024-02-28


# SPOT TESTNET TRADEDE TEGEMINE

In [126]:
import requests
import hashlib
import hmac
import time

# Replace with your Binance API key and secret
api_key = 'd1LUCwf3hCuhtTGfFBf0C16bdD0pIKWP99S4iwFF28xOfHm3g89XCI1QULQnxdR3'
secret_key = 'oI15EZEiIlI916NxTTKxDpU0Ql3tqtlzlafO5gRAbU5dr8Ddjgob6WngRvBQrD74'

# Binance API endpoint for getting account information
url = 'https://testnet.binance.vision/api/v3/account'
# Set the timestamp
timestamp = int(time.time() * 1000)

# Create the query parameters
params = {
    'timestamp': timestamp,
    'recvWindow': 5000,  # Adjust recvWindow if needed, within the specified limits
}

# Create the signature
query_string = '&'.join([f'{key}={value}' for key, value in sorted(params.items())])
signature = hmac.new(secret_key.encode(), query_string.encode(), hashlib.sha256).hexdigest()

# Add the signature to the parameters
params['signature'] = signature

# Set up the request headers
headers = {
    'X-MBX-APIKEY': api_key,
}

# Make the request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    account_info = response.json()
    print(account_info)
else:
    print(f"Error: {response.status_code}, {response.text}")


Error: 400, {"code":-1022,"msg":"Signature for this request is not valid."}


In [128]:
from binance.client import Client
api_key = 'd1LUCwf3hCuhtTGfFBf0C16bdD0pIKWP99S4iwFF28xOfHm3g89XCI1QULQnxdR3'
api_secret = 'oI15EZEiIlI916NxTTKxDpU0Ql3tqtlzlafO5gRAbU5dr8Ddjgob6WngRvBQrD74'

client = Client(api_key, api_secret, testnet=True)


In [146]:
info = client.get_account()
info

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1709843201416,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '7.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.01000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '4027.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '996.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '45.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '177.000000

In [140]:
from binance.enums import *
order = client.create_order(
    symbol='LINKUSDT',
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=2,
    price='21.0')

In [139]:
prices = client.get_all_tickers()
LINKUSDT_data = next(item for item in prices if item['symbol'] == 'LINKUSDT')
LINKUSDT_data


{'symbol': 'LINKUSDT', 'price': '20.06100000'}

In [145]:
orders = client.get_all_orders(symbol='LINKUSDT')
orders

[{'symbol': 'LINKUSDT',
  'orderId': 4598180,
  'orderListId': -1,
  'clientOrderId': 'z3K6HGV0G89afHR7tg69Gp',
  'price': '21.00000000',
  'origQty': '2.00000000',
  'executedQty': '2.00000000',
  'cummulativeQuoteQty': '40.11600000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1709843019123,
  'updateTime': 1709843019123,
  'isWorking': True,
  'workingTime': 1709843019123,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'LINKUSDT',
  'orderId': 4598435,
  'orderListId': -1,
  'clientOrderId': 'PqainVMOXUeY3PAMdVqEIQ',
  'price': '19.00000000',
  'origQty': '2.00000000',
  'executedQty': '2.00000000',
  'cummulativeQuoteQty': '40.12600000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1709843201416,
  'updateTime': 170984320141

In [144]:
order = client.create_order(
    symbol='LINKUSDT',
    side=SIDE_SELL,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=2,
    price='19.0')

# TO DO list


- ~~tee screener, mis võtab nädalaselt graafikult vastavalt elder impulsile~~
- ~~vastavalt bull ja bear setile joonistan esmased graafikud endale välja~~
- ~~salevesta graafikud foldrisse~~
- ~~esita esmane order API-ga~~
- tee esmane web dashboard-i layout
- tee script graafikute salvestamiseks
- pane skript automaatselt tööle
- näita graafikuid dashboardil
- näita dashboardil account infot
- tee orderite tegemine nuppudega dashboardile
- secret_key on vaja panna eraldi faili

